# Streaming Twitter & Translator

## Build Streaming Twitter 

In [ ]:
## The following function is used for listening for twitter terms as well as translating text

In [ ]:
## https://pythondata.com/collecting-storing-tweets-with-python-and-mongodb/

import pymongo
from pymongo import MongoClient
import tweepy
import json

# twitter account keys
CONSUMER_KEY = 'ENTER YOUR KEY HERE'
CONSUMER_SECRET = 'ENTER YOUR KEY HERE'
OAUTH_TOKEN = 'ENTER YOUR KEY HERE'
OAUTH_TOKEN_SECRET = 'ENTER YOUR KEY HERE'

WORDS = ['Hacking', 'Webshell','F5 Exploit','ZeroDay', '0day', 'WPA3 Exploit', 'router exploit', 'Novidade', 'VPNFilter', 'SMB Exploit', 'cisco exploit','استغلال F5', 'قرصنة الكمبيوتر','قذيفة الويب', 'استغلال سيسكو', 'Компьютерный Эксплоит', 'IoT Exploit']
class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        #Connects to your mongoDB and stores the tweet
        try:
            client = MongoClient()
            
            # Use twitterdb database. If it doesn't exist, it will be created.
            db = client['Hacking_Tweets']
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
            #insert the data into the mongoDB into a collection called New_Project
            #if twitter_search doesn't exist, it will be created.
            db.New_Project.insert_one(datajson)
        except Exception as e:
           print(e)
        
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

## Build Translation Function

In [ ]:
### Text Translation Function

In [ ]:
# The following code for tranlation was borrowed from: https://github.com/mouuff/mtranslate/blob/master/mtranslate/core.py

import sys
import re

if (sys.version_info[0] < 3):
    #pythong 2 syntax
    import urllib2
    import urllib
    import HTMLParser
else:
    #python 3 syntax
    import html.parser
    import urllib.request
    import urllib.parse

#this is the user agent string google will see as it is interpreting the tweets    
agent = {'User-Agent':
"Mozilla/4.0 (\
compatible;\
MSIE 6.0;\
Windows NT 5.1;\
SV1;\
.NET CLR 1.1.4322;\
.NET CLR 2.0.50727;\
.NET CLR 3.0.04506.30\
)"}

#these functions are designed to identify if python 2 or python 3 is being used. 
def unescape(text):
    if (sys.version_info[0] < 3):
        parser = HTMLParser.HTMLParser() #python 2
    else:
        parser = html.parser.HTMLParser() #python 3
    return (parser.unescape(text))

def translate(to_translate, to_language="auto", from_language="auto"):
    """Returns the translation using google translate
    you must shortcut the language you define
    (French = fr, English = en, Spanish = es, etc...)
    if not defined it will detect it or use english by default
    Example:
    print(translate("salut tu vas bien?", "en"))
    hello you alright?
    """
    base_link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"
    if (sys.version_info[0] < 3):
        to_translate = urllib.quote_plus(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib2.Request(link, headers=agent)
        raw_data = urllib2.urlopen(request).read()
    else:
        to_translate = urllib.parse.quote(to_translate)
        link = base_link % (to_language, from_language, to_translate)
        request = urllib.request.Request(link, headers=agent)
        raw_data = urllib.request.urlopen(request).read()
    data = raw_data.decode("utf-8")
    expr = r'class="t0">(.*?)<'
    re_result = re.findall(expr, data)
    if (len(re_result) == 0):
        result = ""
    else:
        result = unescape(re_result[0])
    return (result)